<p><img alt="Colaboratory logo" height="45px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>Welcome to Colaboratory!</h1>


Colaboratory is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud.

With Colaboratory you can write and execute code, save and share your analyses, and access powerful computing resources, all for free from your browser.

Initializing the parameters which are the initial stock price, the strike price, the time period, the interest rate and the volatility.

In [0]:
import numpy as np
import pandas as pd

s=1
k=1.1
t=3
r=0.06
no_of_simulations=8
sig=0.5

Intializing the options parameter for some fixed values in the Monte Carlo Simulation

In [220]:
options=[[1,1.09,1.08,1.34],[1,1.16,1.26,1.54],[1,1.22,1.07,1.03],
         [1,0.93,0.97,0.92],[1,1.11,1.56,1.52],[1,0.76,0.77,0.9],
         [1,0.92,0.84,1.01],[1,0.88,1.22,1.34]]
print(list(options))

[[1, 1.09, 1.08, 1.34], [1, 1.16, 1.26, 1.54], [1, 1.22, 1.07, 1.03], [1, 0.93, 0.97, 0.92], [1, 1.11, 1.56, 1.52], [1, 0.76, 0.77, 0.9], [1, 0.92, 0.84, 1.01], [1, 0.88, 1.22, 1.34]]


Randomly initialization of the values for the stock(options variable) for varing number of simulations for different time periods.

In [194]:
options=np.zeros((no_of_simulations,t+1))
for i in range(no_of_simulations):
  options[i][0]=1
print(options)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [154]:
for i in range(no_of_simulations):
  for j in range(1,t+1):
    options[i][j]=np.exp(np.log(options[i][0])+(r-(sig**2/2))+(sig**2)*np.random.normal())
print(options)

[[1, 0.5538621753965774, 1.2848524333075237, 0.7653917070982925], [1, 0.7549544873446716, 1.141438253405036, 0.9547416088827804], [1, 0.9926753940514375, 1.0290927618365344, 0.767357573249674], [1, 1.061362832618001, 1.2309155782161958, 1.0684923523421952], [1, 0.9992194832002755, 1.0593972443713418, 0.8925161446904742], [1, 0.685616035592464, 0.8616581678603924, 0.8523686977761157], [1, 0.5877485535558988, 0.8505028585874168, 0.9961363807549356], [1, 1.1393117221094804, 0.8836273716926246, 1.1180720185069524]]


For each option, the price of the stock is calculated as per the strike price.

In [221]:
prices=np.zeros((no_of_simulations,t))
for i in range(no_of_simulations):
  for j in range(t):
    if(options[i][j+1]>k):
      prices[i][j]=0
    else:
      prices[i][j]=k-options[i][j+1]
print(prices)

[[0.01 0.02 0.  ]
 [0.   0.   0.  ]
 [0.   0.03 0.07]
 [0.17 0.13 0.18]
 [0.   0.   0.  ]
 [0.34 0.33 0.2 ]
 [0.18 0.26 0.09]
 [0.22 0.   0.  ]]


Calculation of the option price using the least squares method using polynomial regression.

In [222]:
vals=np.zeros((no_of_simulations,t))
yr=t-1
while(yr>0):
  stock_values=[]
  for j in range(no_of_simulations):
    if(prices[j][yr-1]!=0):
      arr=[]
      arr.append(j)
      arr.append(prices[j][yr]*np.exp(-r))
      stock_values.append(arr)
  n=len(stock_values)
  one_order_sum=0
  for i in range(len(stock_values)):
    one_order_sum=one_order_sum+options[stock_values[i][0]][yr]
  two_order_sum=0
  for i in range(len(stock_values)):
    two_order_sum=two_order_sum+options[stock_values[i][0]][yr]**2
  three_order_sum=0
  for i in range(len(stock_values)):
    three_order_sum=three_order_sum+options[stock_values[i][0]][yr]**3
  four_order_sum=0
  for i in range(len(stock_values)):
    four_order_sum=four_order_sum+options[stock_values[i][0]][yr]**4
  y_prod=0
  for i in range(len(stock_values)):
    y_prod=y_prod+stock_values[i][1]
  y_prod1=0
  for i in range(len(stock_values)):
    y_prod1=y_prod1+stock_values[i][1]*options[stock_values[i][0]][yr]
  y_prod2=0
  for i in range(len(stock_values)):
    y_prod2=y_prod2+stock_values[i][1]*(options[stock_values[i][0]][yr]**2)
  A=[[n,one_order_sum,two_order_sum],[one_order_sum,two_order_sum,three_order_sum],[two_order_sum,three_order_sum,four_order_sum]]
  #print(A)
  B=[y_prod,y_prod1,y_prod2]
  #print(B)
  C=np.matmul(np.linalg.inv(A),B)
  #print(C)
  c0=C[0]
  c1=C[1]
  c2=C[2]
  vall=[]
  print(C)
  #print(stock_values)
  #print(options)
  for i in range(len(stock_values)):
    new_values=[]
    new_values.append(stock_values[i][0])
    #print(options[stock_values[i][0]][yr])
    new_values.append(c0+c1*options[stock_values[i][0]][yr]+c2*(options[stock_values[i][0]][yr]**2))
    vall.append(new_values)
  #print(vall)
  for i in range(len(vall)):
    #print(vall[i][1])
    #print(prices[stock_values[i][0]][yr-1])
    #print(vall[i][1]>prices[stock_values[i][0]][yr-1])
    if(vall[i][1]>prices[stock_values[i][0]][yr-1]):
      prices[stock_values[i][0]][yr-1]=0
  print(prices)
  yr=yr-1
#print(stock_values) 

[-1.06998766  2.98341063 -1.81357618]
[[0.01 0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.07]
 [0.17 0.13 0.18]
 [0.   0.   0.  ]
 [0.34 0.33 0.2 ]
 [0.18 0.26 0.09]
 [0.22 0.   0.  ]]
[ 2.03751234 -3.3354434   1.35645659]
[[0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.07]
 [0.17 0.13 0.18]
 [0.   0.   0.  ]
 [0.34 0.33 0.2 ]
 [0.18 0.26 0.09]
 [0.22 0.   0.  ]]


In [223]:
for i in range(len(prices)):
  for j in range(len(prices[i])):
    if(prices[i][j]!=0):
      for k in range(j+1,len(prices[i])):
        prices[i][k]=0
print(prices)

[[0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.07]
 [0.17 0.   0.  ]
 [0.   0.   0.  ]
 [0.34 0.   0.  ]
 [0.18 0.   0.  ]
 [0.22 0.   0.  ]]


Calculating the expected option value for a large number of simulations.

In [226]:
opt_value=0
for i in range(len(prices)):
  for j in range(len(prices[i])):
    opt_value=opt_value+prices[i][j]*np.exp(-(j+1)*r)
opt_value=opt_value/no_of_simulations
print(opt_value)

0.11443433004505696


In [0]:
#@title Introducing Colaboratory { display-mode: "form" }
#@markdown This 3-minute video gives an overview of the key features of Colaboratory:
from IPython.display import YouTubeVideo
YouTubeVideo('inN8seMm7UI', width=600, height=400)

## Getting Started

The document you are reading is a  [Jupyter notebook](https://jupyter.org/), hosted in Colaboratory. It is not a static page, but an interactive environment that lets you write and execute code in Python and other languages.

For example, here is a **code cell** with a short Python script that computes a value, stores it in a variable, and prints the result:

In [0]:
seconds_in_a_day = 24 * 60 * 60
seconds_in_a_day

86400

To execute the code in the above cell, select it with a click and then either press the play button to the left of the code, or use the keyboard shortcut "Command/Ctrl+Enter".

All cells modify the same global state, so variables that you define by executing a cell can be used in other cells:

In [0]:
seconds_in_a_week = 7 * seconds_in_a_day
seconds_in_a_week

604800

For more information about working with Colaboratory notebooks, see [Overview of Colaboratory](/notebooks/basic_features_overview.ipynb).


## More Resources

Learn how to make the most of Python, Jupyter, Colaboratory, and related tools with these resources:

### Working with Notebooks in Colaboratory
- [Overview of Colaboratory](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb) 
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas](/notebooks/mlcc/intro_to_pandas.ipynb)
- [Tensorflow concepts](/notebooks/mlcc/tensorflow_programming_concepts.ipynb)
- [First steps with TensorFlow](/notebooks/mlcc/first_steps_with_tensor_flow.ipynb)
- [Intro to neural nets](/notebooks/mlcc/intro_to_neural_nets.ipynb)
- [Intro to sparse data and embeddings](/notebooks/mlcc/intro_to_sparse_data_and_embeddings.ipynb)

### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TensorFlow with TPUs](/notebooks/tpu.ipynb)

## Machine Learning Examples: Seedbank

To see end-to-end examples of the interactive machine learning analyses that Colaboratory makes possible, check out the [Seedbank](https://research.google.com/seedbank/) project.

A few featured examples:

- [Neural Style Transfer](https://research.google.com/seedbank/seed/neural_style_transfer_with_tfkeras): Use deep learning to transfer style between images.
- [EZ NSynth](https://research.google.com/seedbank/seed/ez_nsynth): Synthesize audio with WaveNet auto-encoders.
- [Fashion MNIST with Keras and TPUs](https://research.google.com/seedbank/seed/fashion_mnist_with_keras_and_tpus): Classify fashion-related images with deep learning.
- [DeepDream](https://research.google.com/seedbank/seed/deepdream): Produce DeepDream images from your own photos.
- [Convolutional VAE](https://research.google.com/seedbank/seed/convolutional_vae): Create a generative model of handwritten digits.